## Connect to drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Import packages

In [ ]:
import os
import numpy as np
import pandas as pd
import csv

## Global attributes

In [ ]:
patinets_dataroot = "/content/drive/MyDrive/term project/final_training_data/patient_npl.csv"
normal_dataroot = "/content/drive/MyDrive/term project/final_training_data/ted_npl_result.csv"

#nlp result table
txt_df = pd.read_csv(normal_dataroot)
txt_df[145:150]

,name,words_number,SPACE,ADV,VERB,ADP,DET,NOUN,ADJ,PUNCT,...,Punctuation_R,hestitation_word_R,lemma_number_R,person_singular_verbs_R,misspell_R,time_spec_R,spec_R,neg_word_R,content_R,function_R
145,DanPhillips_2010X.sph,606.0,1.0,54.0,64.0,58.0,71.0,117.0,25.0,54.0,...,0.089109,0.006601,0.356436,0.013201,0.009901,0.00000,0.021452,0.450495,0.415842,NaN
146,DaveEggers_2008.sph,678.0,1.0,45.0,78.0,68.0,46.0,103.0,31.0,92.0,...,0.135693,0.004425,0.309735,0.014749,0.002950,0.00295,0.044248,0.424779,0.370206,NaN
147,DavidAgus_2009P.sph,563.0,1.0,25.0,73.0,58.0,48.0,100.0,28.0,50.0,...,0.088810,0.000000,0.351687,0.007105,0.000000,0.00000,0.030195,0.452931,0.413854,NaN
148,DavidBismark_2010G.sph,488.0,1.0,28.0,79.0,42.0,42.0,87.0,29.0,45.0,...,0.092213,0.006148,0.362705,0.002049,0.002049,0.00000,0.004098,0.506148,0.358607,NaN
149,DavidBlaine_2009P.sph,523.0,1.0,34.0,72.0,53.0,32.0,76.0,21.0,46.0,...,0.087954,0.001912,0.388145,0.042065,0.005736,0.00956,0.036329,0.434034,0.407266,NaN


## MFCC data extraction

In [ ]:
class data_process:
    def __init__(self):
        self.labels = []
        self.mean_features = []
        self.std_features = []
        self.var_features = []
        self.min_features = []
        self.max_features = []

    def cal_mfcc_features(self, file_name, df):
        self.mean_features.append(df.mean(axis = 0))   # mean value of each frame
        self.std_features.append(df.std(axis = 0))     # standard deviation of each frame
        self.var_features.append(df.var(axis = 0))     # variance of each frame
        self.min_features.append(df.min(axis = 0))     # minimum of each frame
        self.max_features.append(df.max(axis = 0))     # maximum of each frame


## Read in datafiles


In [ ]:
patient_info = pd.read_csv(patinets_dataroot)

dp = data_process()
print(patient_info.shape[0])

audio_names = []
mean_names = ["mean_0", "mean_1", "mean_2", "mean_3", "mean_4", "mean_5", "mean_6", "mean_7", "mean_8", "mean_9", "mean_10", "mean_11"]
std_names = ["std_0", "std_1", "std_2", "std_3", "std_4", "std_5", "std_6", "std_7", "std_8", "std_9", "std_10", "std_11"]
var_names = ["var_0", "var_1", "var_2", "var_3", "var_4", "var_5", "var_6", "var_7", "var_8", "var_9", "var_10", "var_11"]
min_names = ["min_0", "min_1", "min_2", "min_3", "min_4", "min_5", "min_6", "min_7", "min_8", "min_9", "min_10", "min_11"]
max_names = ["max_0", "max_1", "max_2", "max_3", "max_4", "max_5", "max_6", "max_7", "max_8", "max_9", "max_10", "max_11"]

#MFCC result
for i in range(patient_info.shape[0]):
    aud_df = pd.read_csv(f"/content/drive/MyDrive/term project/final_training_data/482_csv_MFCC/{patient_info.iloc[i,0]}.csv", header=None)
    audio_names.append(patient_info.iloc[i,0])
    dp.cal_mfcc_features(patient_info.iloc[i,0], aud_df)


mean_df = pd.DataFrame(dp.mean_features)
mean_df.columns = mean_names
mean_df["name"] = audio_names
mean_df = mean_df.reindex(["name", "mean_0", "mean_1", "mean_2", "mean_3", "mean_4", "mean_5", "mean_6", "mean_7", "mean_8", "mean_9", "mean_10", "mean_11"], axis=1)

std_df = pd.DataFrame(dp.std_features)
std_df.columns = std_names
std_df["name"] = audio_names

var_df = pd.DataFrame(dp.var_features)
var_df.columns = var_names
var_df["name"] = audio_names

min_df = pd.DataFrame(dp.min_features)
min_df.columns = min_names
min_df["name"] = audio_names

max_df = pd.DataFrame(dp.max_features)
max_df.columns = max_names
max_df["name"] = audio_names

mfcc_df = pd.merge(mean_df, std_df, how="right", on = "name")
mfcc_df = pd.merge(mfcc_df, var_df, how="right", on = "name")
mfcc_df = pd.merge(mfcc_df, min_df, how="right", on = "name")
mfcc_df = pd.merge(mfcc_df, max_df, how="right", on = "name")
mfcc_df


526


,name,mean_0,mean_1,mean_2,mean_3,mean_4,mean_5,mean_6,mean_7,mean_8,...,max_2,max_3,max_4,max_5,max_6,max_7,max_8,max_9,max_10,max_11
0,Baycrest11633,0.854198,0.077458,-0.127762,0.040351,-1.008094,-0.486217,-0.578463,-0.863399,-0.644760,...,21.488652,18.165062,11.832625,9.417896,11.446426,7.816877,5.399572,6.187289,7.383444,6.679798
1,Baycrest11634,0.541553,-0.540911,-0.571749,-0.418698,-0.527357,-1.110842,-0.188168,-0.894949,-0.013404,...,22.219027,16.285212,14.096027,8.841052,12.630060,8.252337,8.813189,7.760667,9.050715,7.782302
2,Baycrest11976,0.723079,-0.774634,-0.001862,-0.853804,0.115562,-0.789923,-0.286295,-1.087737,-0.497240,...,22.188276,16.193934,17.533318,10.466017,11.771722,8.965360,7.714233,8.847869,7.822739,6.995494
3,Baycrest12257,0.494907,-0.788060,-0.578600,-0.714923,-0.361604,-1.022182,-0.077619,-1.467050,0.036221,...,22.161707,14.532452,15.230025,10.634626,14.155958,9.152628,9.955507,9.015206,8.979043,8.721945
4,Baycrest12813,1.160238,-0.695389,-0.333076,-0.647016,-0.582254,-0.722315,0.035269,-1.872454,-0.376687,...,20.807168,12.900416,15.716621,9.559799,14.857382,7.704551,9.116838,8.673863,10.162672,7.317368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,Pitt_sentence_702-0,0.583718,-1.021799,2.516685,-0.902062,1.494482,-1.409396,0.753968,-2.190949,1.044270,...,15.936191,10.309797,7.600101,6.668975,8.829612,4.222050,7.787092,4.664528,4.836586,2.550961
522,Pitt_sentence_703-0,5.956598,-1.412940,-1.041006,-1.947886,1.576066,-2.038081,1.070953,-1.308377,0.986133,...,14.115372,6.874049,8.619025,5.560655,6.519713,4.481349,7.780946,5.865544,4.506490,4.413810
523,Pitt_sentence_704-0,0.154768,-1.872016,2.083084,-1.928786,0.965853,-2.285675,0.829735,-1.899469,0.110970,...,16.188278,12.060864,8.756555,5.912059,6.956959,5.439173,5.616610,5.149533,5.958505,5.284714
524,Pitt_sentence_707-0,3.048985,-0.706024,0.255343,-2.195804,0.759685,-2.643669,0.836204,-1.732588,-0.122005,...,11.792063,8.649033,7.985114,3.210298,8.973679,5.437198,4.555414,4.664717,4.007050,5.008276


## merge mfcc & txt data

In [ ]:
txt_df = pd.read_csv(patinets_dataroot)
merged_patient_df = pd.merge(mfcc_df, txt_df, how="right", on = "name")
merged_patient_df["AD_diagnose"] = np.ones(patient_info.shape[0], dtype = int)


print(merged_patient_df.shape)
merged_patient_df

(526, 120)


,name,mean_0,mean_1,mean_2,mean_3,mean_4,mean_5,mean_6,mean_7,mean_8,...,hestitation_word_R,lemma_number_R,person_singular_verbs_R,misspell_R,time_spec_R,spec_R,neg_word_R,content_R,function_R,AD_diagnose
0,Baycrest11633,0.854198,0.077458,-0.127762,0.040351,-1.008094,-0.486217,-0.578463,-0.863399,-0.644760,...,0.019194,0.357006,0.021113,0.000000,0.000000,0.017274,0.404990,0.431862,NaN,1
1,Baycrest11634,0.541553,-0.540911,-0.571749,-0.418698,-0.527357,-1.110842,-0.188168,-0.894949,-0.013404,...,0.013349,0.197941,0.017925,0.003051,0.004195,0.020214,0.438596,0.490465,NaN,1
2,Baycrest11976,0.723079,-0.774634,-0.001862,-0.853804,0.115562,-0.789923,-0.286295,-1.087737,-0.497240,...,0.019862,0.261658,0.014680,0.006045,0.003454,0.018135,0.424870,0.487910,NaN,1
3,Baycrest12257,0.494907,-0.788060,-0.578600,-0.714923,-0.361604,-1.022182,-0.077619,-1.467050,0.036221,...,0.015808,0.178410,0.023487,0.009485,0.004065,0.024842,0.374435,0.392954,NaN,1
4,Baycrest12813,1.160238,-0.695389,-0.333076,-0.647016,-0.582254,-0.722315,0.035269,-1.872454,-0.376687,...,0.009127,0.176984,0.023810,0.006746,0.000794,0.013889,0.379365,0.408333,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,Pitt_sentence_702-0,0.583718,-1.021799,2.516685,-0.902062,1.494482,-1.409396,0.753968,-2.190949,1.044270,...,0.000000,0.630769,0.015385,0.000000,0.000000,0.030769,0.369231,0.415385,NaN,1
522,Pitt_sentence_703-0,5.956598,-1.412940,-1.041006,-1.947886,1.576066,-2.038081,1.070953,-1.308377,0.986133,...,0.052632,0.466165,0.030075,0.000000,0.000000,0.000000,0.308271,0.421053,NaN,1
523,Pitt_sentence_704-0,0.154768,-1.872016,2.083084,-1.928786,0.965853,-2.285675,0.829735,-1.899469,0.110970,...,0.026316,0.442982,0.021930,0.004386,0.000000,0.030702,0.403509,0.381579,NaN,1
524,Pitt_sentence_707-0,3.048985,-0.706024,0.255343,-2.195804,0.759685,-2.643669,0.836204,-1.732588,-0.122005,...,0.000000,0.673077,0.019231,0.000000,0.019231,0.057692,0.403846,0.423077,NaN,1


In [ ]:
drop_cols = ['SPACE', 'ADV', 'VERB', 'ADP', 'DET', 'NOUN', 'ADJ', 'PUNCT', 'INTJ', 'NUM', 'PRON', 'AUX', 'CCONJ', 'PART', 'PROPN', 'SCONJ', 'CONJ', 'Punctuation', 'hestitation_word', 'lemma_number', 'most_frequent', 'noun_chunk', 'person_singular_verbs', 'misspell', 'time_spec', 'spec', 'sentence', 'neg_word', 'content', 'function', 'function_R']
merged_patient_df = merged_patient_df.drop(drop_cols, axis = 1)

for i in range(525):
    if (MCI_df["label"][i] == 0):
        merged_patient_df.at[i, "AD_diagnose"] = 2

merged_patient_df.to_csv('/content/drive/MyDrive/term project/final_training_data/MCI_patient.csv', index=False)

(526, 89)


## Write patient to file

In [ ]:
csv_file_path = '/content/drive/MyDrive/term project/final_training_data/MCI_patient_data.csv'
merged_patient_df.to_csv(csv_file_path, index=False)

## Normal people

In [ ]:
normal_info = pd.read_csv(normal_dataroot)
normal_info.drop(normal_info[normal_info["name"] == "list.txt"].index, inplace = True)
normal_info.drop(normal_info[normal_info["name"] == "name.bat"].index, inplace = True)

dp = data_process()
print(normal_info.shape[0])


normal_audio_names = []
mean_names = ["mean_0", "mean_1", "mean_2", "mean_3", "mean_4", "mean_5", "mean_6", "mean_7", "mean_8", "mean_9", "mean_10", "mean_11"]
std_names = ["std_0", "std_1", "std_2", "std_3", "std_4", "std_5", "std_6", "std_7", "std_8", "std_9", "std_10", "std_11"]
var_names = ["var_0", "var_1", "var_2", "var_3", "var_4", "var_5", "var_6", "var_7", "var_8", "var_9", "var_10", "var_11"]
min_names = ["min_0", "min_1", "min_2", "min_3", "min_4", "min_5", "min_6", "min_7", "min_8", "min_9", "min_10", "min_11"]
max_names = ["max_0", "max_1", "max_2", "max_3", "max_4", "max_5", "max_6", "max_7", "max_8", "max_9", "max_10", "max_11"]

for i in range(normal_info.shape[0]):
    normal_aud_df = pd.read_csv(f"/content/drive/MyDrive/term project/final_training_data/normal_output/unnoise_{normal_info.iloc[i,0][:-4]}.csv", header=None)
    normal_audio_names.append(normal_info.iloc[i,0])
    dp.cal_mfcc_features(normal_info.iloc[i,0], normal_aud_df)

normal_mean_df = pd.DataFrame(dp.mean_features)
normal_mean_df.columns = mean_names
normal_mean_df["name"] = normal_audio_names
normal_mean_df = normal_mean_df.reindex(["name", "mean_0", "mean_1", "mean_2", "mean_3", "mean_4", "mean_5", "mean_6", "mean_7", "mean_8", "mean_9", "mean_10", "mean_11"], axis=1)

normal_std_df = pd.DataFrame(dp.std_features)
normal_std_df.columns = std_names
normal_std_df["name"] = normal_audio_names

normal_var_df = pd.DataFrame(dp.var_features)
normal_var_df.columns = var_names
normal_var_df["name"] = normal_audio_names

normal_min_df = pd.DataFrame(dp.min_features)
normal_min_df.columns = min_names
normal_min_df["name"] = normal_audio_names

normal_max_df = pd.DataFrame(dp.max_features)
normal_max_df.columns = max_names
normal_max_df["name"] = normal_audio_names

normal_mfcc_df = pd.merge(normal_mean_df, normal_std_df, how="right", on = "name")
normal_mfcc_df = pd.merge(normal_mfcc_df, normal_var_df, how="right", on = "name")
normal_mfcc_df = pd.merge(normal_mfcc_df, normal_min_df, how="right", on = "name")
normal_mfcc_df = pd.merge(normal_mfcc_df, normal_max_df, how="right", on = "name")
normal_mfcc_df

774


,name,mean_0,mean_1,mean_2,mean_3,mean_4,mean_5,mean_6,mean_7,mean_8,...,max_2,max_3,max_4,max_5,max_6,max_7,max_8,max_9,max_10,max_11
0,AaronHuey_2010X.sph,-2.100777,-1.684115,2.544570,-1.989832,-1.695910,-1.573015,-1.261112,0.145461,-0.405730,...,30.810466,14.380498,15.529417,13.795725,14.881015,13.453976,11.374592,11.262426,8.080116,8.143906
1,AdamGrosser_2007.sph,2.026874,-2.970676,-0.211229,-1.020247,-2.337118,-1.624654,-1.856880,-0.892982,0.208198,...,26.665672,15.547970,14.335839,12.837151,15.321939,13.935210,11.167987,14.009330,9.046028,7.619562
2,AdamSadowsky_2010X.sph,-1.066398,-2.491490,0.153002,-1.590363,-2.968878,-1.996254,-1.974280,-0.964948,0.203181,...,26.763144,16.491785,13.469095,15.084549,11.301493,12.721368,12.628423,10.225828,9.317845,8.652215
3,AdamSavage_2008P.sph,-0.195557,0.513080,0.371363,-2.126603,-1.885724,-1.882342,-1.734679,-1.513676,-0.424407,...,27.653897,16.195575,17.356928,20.394594,10.861738,14.059675,14.002002,13.475784,7.174346,8.707660
4,AditiShankardass_2009I.sph,-3.756178,-3.823526,-0.622556,-2.488703,-2.752145,-1.834806,-0.154068,-0.948735,-0.054191,...,21.303022,16.167965,21.192390,12.403212,13.093831,14.341368,14.033656,8.383592,7.912875,9.329742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
769,YvesBehar_2009.sph,0.353279,-1.622894,0.338187,-1.117405,-1.788555,-1.247814,-1.475865,-0.513751,0.452092,...,25.431162,15.830408,15.020955,14.489093,16.833880,12.388460,12.315859,8.944325,9.249893,8.842881
770,ZainabSalbi_2010G.sph,-0.927350,-1.675533,-0.847566,-1.682460,-1.089285,-1.277828,-1.011582,-1.385820,-1.025862,...,24.045691,21.954162,16.008326,14.570586,12.149753,10.984533,10.340586,8.435837,8.111250,7.489990
771,ZeFrank_2004.sph,-0.619673,-4.062159,-0.771935,-2.115899,-2.626143,-1.624953,-1.490587,0.174504,-0.223774,...,27.147582,16.074433,18.011855,15.976929,16.086246,16.662259,11.409941,11.021287,10.149223,8.898061
772,ZeFrank_2010G.sph,-1.436657,-1.978155,-1.599265,-1.424246,-1.760718,-1.980226,-1.442452,-1.025259,-0.682320,...,23.834778,13.512251,12.195885,13.288000,9.850386,10.677720,9.868671,9.291246,9.178457,7.934675


In [ ]:
print(normal_info.shape)
normal_txt_df = pd.read_csv(normal_dataroot)
merged_normal_df = pd.merge(normal_mfcc_df, normal_info, how="right", on = "name")
merged_normal_df["AD_diagnose"] = np.zeros(normal_info.shape[0], dtype = int)


print(merged_normal_df.shape)
merged_normal_df

(774, 59)
(774, 120)


,name,mean_0,mean_1,mean_2,mean_3,mean_4,mean_5,mean_6,mean_7,mean_8,...,hestitation_word_R,lemma_number_R,person_singular_verbs_R,misspell_R,time_spec_R,spec_R,neg_word_R,content_R,function_R,AD_diagnose
0,AaronHuey_2010X.sph,-2.100777,-1.684115,2.544570,-1.989832,-1.695910,-1.573015,-1.261112,0.145461,-0.405730,...,0.000000,0.423423,0.003604,0.057658,0.000000,0.072072,0.389189,0.383784,NaN,0
1,AdamGrosser_2007.sph,2.026874,-2.970676,-0.211229,-1.020247,-2.337118,-1.624654,-1.856880,-0.892982,0.208198,...,0.000000,0.365079,0.001764,0.008818,0.007055,0.045855,0.430335,0.386243,NaN,0
2,AdamSadowsky_2010X.sph,-1.066398,-2.491490,0.153002,-1.590363,-2.968878,-1.996254,-1.974280,-0.964948,0.203181,...,0.005386,0.396768,0.000000,0.000000,0.000000,0.019749,0.461400,0.366248,NaN,0
3,AdamSavage_2008P.sph,-0.195557,0.513080,0.371363,-2.126603,-1.885724,-1.882342,-1.734679,-1.513676,-0.424407,...,0.001570,0.423862,0.025118,0.028257,0.000000,0.029827,0.453689,0.458399,NaN,0
4,AditiShankardass_2009I.sph,-3.756178,-3.823526,-0.622556,-2.488703,-2.752145,-1.834806,-0.154068,-0.948735,-0.054191,...,0.000000,0.427646,0.004320,0.028078,0.000000,0.038877,0.481641,0.356371,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
769,YvesBehar_2009.sph,0.353279,-1.622894,0.338187,-1.117405,-1.788555,-1.247814,-1.475865,-0.513751,0.452092,...,0.000000,0.487437,0.005025,0.000000,0.010050,0.035176,0.432161,0.391960,NaN,0
770,ZainabSalbi_2010G.sph,-0.927350,-1.675533,-0.847566,-1.682460,-1.089285,-1.277828,-1.011582,-1.385820,-1.025862,...,0.000000,0.407792,0.023377,0.002597,0.000000,0.015584,0.415584,0.462338,NaN,0
771,ZeFrank_2004.sph,-0.619673,-4.062159,-0.771935,-2.115899,-2.626143,-1.624953,-1.490587,0.174504,-0.223774,...,0.003752,0.465291,0.011257,0.075047,0.000000,0.033771,0.363977,0.371482,NaN,0
772,ZeFrank_2010G.sph,-1.436657,-1.978155,-1.599265,-1.424246,-1.760718,-1.980226,-1.442452,-1.025259,-0.682320,...,0.013233,0.395085,0.007561,0.015123,0.001890,0.026465,0.413989,0.387524,NaN,0


## Write normal people to file

In [ ]:
csv_file_path = '/content/drive/MyDrive/term project/final_training_data/merged_normal_data.csv'
merged_normal_df.to_csv(csv_file_path, index=False)

## merge whole dataset

In [ ]:
final_dataset = pd.concat([merged_patient_df, merged_normal_df])
final_dataset.to_csv('/content/drive/MyDrive/term project/final_training_data/merged_data.csv', index=False)

## Normalized merged dataset

In [ ]:
# NLP features
# ['words_number', 'SPACE', 'ADV', 'VERB', 'ADP', 'DET', 'NOUN', 'ADJ', 'PUNCT', 'INTJ', 'NUM', 'PRON', 'AUX', 'CCONJ', 'PART', 'PROPN', 'SCONJ', 'CONJ', 'Punctuation', 'hestitation_word', 'lemma_number', 'most_frequent', 'noun_chunk', 'person_singular_verbs', 'misspell', 'time_spec', 'spec', 'sentence', 'neg_word', 'content', 'function', 'SPACE_R', 'ADV_R', 'VERB_R', 'ADP_R', 'DET_R', 'NOUN_R', 'ADJ_R', 'PUNCT_R', 'INTJ_R', 'NUM_R', 'PRON_R', 'AUX_R', 'CCONJ_R', 'PART_R', 'PROPN_R', 'SCONJ_R', 'CONJ_R', 'Punctuation_R', 'hestitation_word_R', 'lemma_number_R', 'person_singular_verbs_R', 'misspell_R', 'time_spec_R', 'spec_R', 'neg_word_R', 'content_R', 'function_R']

# Dataset without not normalized data
drop_cols = ['SPACE', 'ADV', 'VERB', 'ADP', 'DET', 'NOUN', 'ADJ', 'PUNCT', 'INTJ', 'NUM', 'PRON', 'AUX', 'CCONJ', 'PART', 'PROPN', 'SCONJ', 'CONJ', 'Punctuation', 'hestitation_word', 'lemma_number', 'most_frequent', 'noun_chunk', 'person_singular_verbs', 'misspell', 'time_spec', 'spec', 'sentence', 'neg_word', 'content', 'function', 'function_R']
final_dataset_normalized = final_dataset.drop(drop_cols, axis = 1)
final_dataset_normalized.to_csv('/content/drive/MyDrive/term project/final_training_data/normalized_merged_data.csv', index=False)
final_dataset_normalized

,name,mean_0,mean_1,mean_2,mean_3,mean_4,mean_5,mean_6,mean_7,mean_8,...,Punctuation_R,hestitation_word_R,lemma_number_R,person_singular_verbs_R,misspell_R,time_spec_R,spec_R,neg_word_R,content_R,AD_diagnose
0,Baycrest11633,0.854198,0.077458,-0.127762,0.040351,-1.008094,-0.486217,-0.578463,-0.863399,-0.644760,...,0.109405,0.019194,0.357006,0.021113,0.000000,0.000000,0.017274,0.404990,0.431862,1
1,Baycrest11634,0.541553,-0.540911,-0.571749,-0.418698,-0.527357,-1.110842,-0.188168,-0.894949,-0.013404,...,0.004195,0.013349,0.197941,0.017925,0.003051,0.004195,0.020214,0.438596,0.490465,1
2,Baycrest11976,0.723079,-0.774634,-0.001862,-0.853804,0.115562,-0.789923,-0.286295,-1.087737,-0.497240,...,0.016408,0.019862,0.261658,0.014680,0.006045,0.003454,0.018135,0.424870,0.487910,1
3,Baycrest12257,0.494907,-0.788060,-0.578600,-0.714923,-0.361604,-1.022182,-0.077619,-1.467050,0.036221,...,0.165312,0.015808,0.178410,0.023487,0.009485,0.004065,0.024842,0.374435,0.392954,1
4,Baycrest12813,1.160238,-0.695389,-0.333076,-0.647016,-0.582254,-0.722315,0.035269,-1.872454,-0.376687,...,0.153968,0.009127,0.176984,0.023810,0.006746,0.000794,0.013889,0.379365,0.408333,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
769,YvesBehar_2009.sph,0.353279,-1.622894,0.338187,-1.117405,-1.788555,-1.247814,-1.475865,-0.513751,0.452092,...,0.125628,0.000000,0.487437,0.005025,0.000000,0.010050,0.035176,0.432161,0.391960,0
770,ZainabSalbi_2010G.sph,-0.927350,-1.675533,-0.847566,-1.682460,-1.089285,-1.277828,-1.011582,-1.385820,-1.025862,...,0.088312,0.000000,0.407792,0.023377,0.002597,0.000000,0.015584,0.415584,0.462338,0
771,ZeFrank_2004.sph,-0.619673,-4.062159,-0.771935,-2.115899,-2.626143,-1.624953,-1.490587,0.174504,-0.223774,...,0.138837,0.003752,0.465291,0.011257,0.075047,0.000000,0.033771,0.363977,0.371482,0
772,ZeFrank_2010G.sph,-1.436657,-1.978155,-1.599265,-1.424246,-1.760718,-1.980226,-1.442452,-1.025259,-0.682320,...,0.111531,0.013233,0.395085,0.007561,0.015123,0.001890,0.026465,0.413989,0.387524,0


In [ ]:
original_dataset = "/content/drive/MyDrive/term project/final_training_data/normalized_merged_data.csv"
MCI_info = "/content/drive/MyDrive/term project/final_training_data/patient_info.csv"
MCI_df = pd.read_csv(MCI_info)
original_df = pd.read_csv(original_dataset)
for i in range(525):
    if (MCI_df["label"][i] == 0):
        original_df.at[i, "AD_diagnose"] = 2
original_df.to_csv('/content/drive/MyDrive/term project/final_training_data/MCI_data.csv', index=False)